# Predict interaction energies for missing AAs

Here we use ML to fill the interaction energies for the missing aminoacids:

In [1]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem import Descriptors

from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error, r2_score

In [2]:
monomer_smiles = {
    'A': 'CC(C(=O)O)N',  # protein
    'R': 'C(CC(C(=O)O)N)CN=C(N)N',
    'N': 'C(C(C(=O)O)N)C(=O)N',
    'D': 'C(C(C(=O)O)N)C(=O)O',
    'C': 'C(C(C(=O)O)N)S',
    'Q': 'C(CC(=O)N)C(C(=O)O)N',
    'E': 'C(CC(=O)O)C(C(=O)O)N',
    'G': 'C(C(=O)O)N',
    'H': 'C1=C(NC=N1)CC(C(=O)O)N',
    'I': 'CCC(C)C(C(=O)O)N',
    'L': 'CC(C)CC(C(=O)O)N',
    'K': 'C(CCN)CC(C(=O)O)N',
    'M': 'CSCCC(C(=O)O)N',
    'F': 'C1=CC=C(C=C1)CC(C(=O)O)N',
    'P': 'C1CC(NC1)C(=O)O',
    'S': 'C(C(C(=O)O)N)O',
    'T': 'CC(C(C(=O)O)N)O',
    'W': 'C1=CC=C2C(=C1)C(=CN2)CC(C(=O)O)N',
    'Y': 'C1=CC(=CC=C1CC(C(=O)O)N)O',
    'V': 'CC(C)C(C(=O)O)N',
    'O': 'CC1CC=NC1C(=O)NCCCCC(C(=O)O)N',
    'U': 'C(C(C(=O)O)N)[Se]'
}

In [3]:
df = pd.read_csv('data/energies.csv')
df

,Aminoacid,"Mg interaction energy, eV","Ca interaction energy, eV","Ba interaction energy, eV","Mg interaction energy, charged sidechains, eV","Ca interaction energy, charged sidechains, eV","Ba interaction energy, charged sidechains, eV"
0,A,-0.138234,-0.118715,-0.100234,-0.138234,-0.118715,-0.100234
1,R,-3.225336,-2.127782,-1.599040,4.481403,4.390957,4.303165
2,N,-1.800469,-1.256246,-0.895895,-1.800469,-1.256246,-0.895895
3,D,-1.326600,-0.831851,-0.510109,-8.705812,-8.138319,-7.615657
4,C,-0.880283,-0.458234,-0.233128,-0.880283,-0.458234,-0.233128
5,Q,-2.364570,-1.721022,-1.321998,-2.364570,-1.721022,-1.321998
6,E,-1.717493,-1.120843,-0.900507,-8.961262,-8.330724,-7.762967
7,G,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,H,-2.353829,-1.601587,-1.211088,5.551417,5.389953,5.226539
9,I,-0.638260,-0.347956,-0.270345,-0.638260,-0.347956,-0.270345


In [4]:
# add smiles columns
df['smiles'] = df['Aminoacid'].map(monomer_smiles)

# addend for new AAs
additional_aa = [['O', 'CC1CC=NC1C(=O)NCCCCC(C(=O)O)N'],
                 ['U', 'C(C(C(=O)O)N)[Se]']]
df_new = pd.DataFrame(additional_aa, columns=['Aminoacid', 'smiles'])
df_new

,Aminoacid,smiles
0,O,CC1CC=NC1C(=O)NCCCCC(C(=O)O)N
1,U,C(C(C(=O)O)N)[Se]


In [5]:
data = pd.concat([df, df_new], axis=0).reset_index(drop=True)
data

,Aminoacid,"Mg interaction energy, eV","Ca interaction energy, eV","Ba interaction energy, eV","Mg interaction energy, charged sidechains, eV","Ca interaction energy, charged sidechains, eV","Ba interaction energy, charged sidechains, eV",smiles
0,A,-0.138234,-0.118715,-0.100234,-0.138234,-0.118715,-0.100234,CC(C(=O)O)N
1,R,-3.225336,-2.127782,-1.599040,4.481403,4.390957,4.303165,C(CC(C(=O)O)N)CN=C(N)N
2,N,-1.800469,-1.256246,-0.895895,-1.800469,-1.256246,-0.895895,C(C(C(=O)O)N)C(=O)N
3,D,-1.326600,-0.831851,-0.510109,-8.705812,-8.138319,-7.615657,C(C(C(=O)O)N)C(=O)O
4,C,-0.880283,-0.458234,-0.233128,-0.880283,-0.458234,-0.233128,C(C(C(=O)O)N)S
5,Q,-2.364570,-1.721022,-1.321998,-2.364570,-1.721022,-1.321998,C(CC(=O)N)C(C(=O)O)N
6,E,-1.717493,-1.120843,-0.900507,-8.961262,-8.330724,-7.762967,C(CC(=O)O)C(C(=O)O)N
7,G,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,C(C(=O)O)N
8,H,-2.353829,-1.601587,-1.211088,5.551417,5.389953,5.226539,C1=C(NC=N1)CC(C(=O)O)N
9,I,-0.638260,-0.347956,-0.270345,-0.638260,-0.347956,-0.270345,CCC(C)C(C(=O)O)N


## Adding rdkit descriptors

To fill missing data we want to use ML algorithms. So we need more descriptors to find distance between data points. Thta's why we are adding new descriptors, later we will drop them.

In [6]:
descriptor_names = list(rdMolDescriptors.Properties.GetAvailableProperties())
get_descriptors = rdMolDescriptors.Properties(descriptor_names)
num_descriptors = len(descriptor_names)

# Initiallization of the empty matrix
descriptors_set = np.empty((0, num_descriptors), float)


for _, row in data.iterrows():
    smiles = row['smiles']
    molecule = Chem.MolFromSmiles(smiles)

    if molecule is not None:
        descriptors = np.array(get_descriptors.ComputeProperties(molecule)).reshape((-1, num_descriptors))
        descriptors_set = np.append(descriptors_set, descriptors, axis=0)

# Creating DataFrame with descriptors
df_descriptors = pd.DataFrame(descriptors_set, columns=descriptor_names)

data = pd.concat([data, df_descriptors], axis=1)

In [7]:
data

,Aminoacid,"Mg interaction energy, eV","Ca interaction energy, eV","Ba interaction energy, eV","Mg interaction energy, charged sidechains, eV","Ca interaction energy, charged sidechains, eV","Ba interaction energy, charged sidechains, eV",smiles,exactmw,amw,...,chi0n,chi1n,chi2n,chi3n,chi4n,hallKierAlpha,kappa1,kappa2,kappa3,Phi
0,A,-0.138234,-0.118715,-0.100234,-0.138234,-0.118715,-0.100234,CC(C(=O)O)N,89.047678,89.094,...,3.510162,1.627090,0.389528,0.389528,0.000000,-0.570000,5.430000,1.767634,1.721545,1.599709
1,R,-3.225336,-2.127782,-1.599040,4.481403,4.390957,4.303165,C(CC(C(=O)O)N)CN=C(N)N,174.111676,174.204,...,6.733397,3.575173,1.284817,1.284817,0.642832,-1.300000,10.700000,5.363379,5.482230,4.782347
2,N,-1.800469,-1.256246,-0.895895,-1.800469,-1.256246,-0.895895,C(C(C(=O)O)N)C(=O)N,132.053492,132.119,...,4.702868,2.304340,0.738295,0.738295,0.304056,-1.100000,7.900000,3.032307,3.479710,2.661691
3,D,-1.326600,-0.831851,-0.510109,-8.705812,-8.138319,-7.615657,C(C(C(=O)O)N)C(=O)O,133.037508,133.103,...,4.572731,2.239272,0.711731,0.711731,0.275438,-1.100000,7.900000,3.032307,3.479710,2.661691
4,C,-0.880283,-0.458234,-0.233128,-0.880283,-0.458234,-0.233128,C(C(C(=O)O)N)S,121.019749,121.161,...,3.664483,1.774215,0.513894,0.513894,0.078093,-0.220000,6.780000,2.872925,2.472042,2.782633
5,Q,-2.364570,-1.721022,-1.321998,-2.364570,-1.721022,-1.321998,C(CC(=O)N)C(C(=O)O)N,146.069142,146.146,...,5.409975,2.804340,1.018939,1.018939,0.421236,-1.100000,8.900000,3.837557,3.848566,3.415426
6,E,-1.717493,-1.120843,-0.900507,-8.961262,-8.330724,-7.762967,C(CC(=O)O)C(C(=O)O)N,147.053158,147.130,...,5.279838,2.739272,0.986405,0.986405,0.402453,-1.100000,8.900000,3.837557,3.848566,3.415426
7,G,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,C(C(=O)O)N,75.032028,75.067,...,2.639919,1.189533,0.174620,0.174620,0.000000,-0.570000,4.430000,1.721545,3.430000,1.525289
8,H,-2.353829,-1.601587,-1.211088,5.551417,5.389953,5.226539,C1=C(NC=N1)CC(C(=O)O)N,155.069477,155.157,...,5.819183,3.155289,1.311877,1.311877,0.720551,-1.360000,7.743734,3.156504,2.047487,2.222102
9,I,-0.638260,-0.347956,-0.270345,-0.638260,-0.347956,-0.270345,CCC(C)C(C(=O)O)N,131.094629,131.175,...,5.794619,3.075778,1.542155,1.542155,0.497328,-0.570000,8.430000,3.454517,2.463571,3.235731


# Filling 'Ca interaction energy' column

In [8]:
np.random.seed(1234)

# Define the descriptors and target column
drop_cols = list(data.columns[:8])
descriptors = data.columns.difference(drop_cols)
target_columns = list(data.columns[1:7])

for target_column in target_columns:
    
    # Split the data into training and testing sets
    train_data = data.iloc[:-2]  # Exclude the last two rows with missing values
    test_data = data.iloc[-2:]

    # Create a random forest model to predict missing values
    model = RandomForestRegressor()

    # Train the model on data without missing values in the target column
    model.fit(train_data[descriptors], train_data[target_column])

    # Predict the missing values
    predicted_values = model.predict(test_data[descriptors])

    # Fill in the missing values in the original dataset
    data.loc[data.index[-2:], target_column] = predicted_values

In [9]:
# drop columns
res = data.iloc[:, :7]

# substitute charged sidechains values for U
idxs = res['Aminoacid'] == 'U' # uncharged system
for M in ('Mg', 'Ca', 'Ba'):
    res.loc[idxs, f'{M} interaction energy, charged sidechains, eV'] = res.loc[idxs, f'{M} interaction energy, eV']

res

,Aminoacid,"Mg interaction energy, eV","Ca interaction energy, eV","Ba interaction energy, eV","Mg interaction energy, charged sidechains, eV","Ca interaction energy, charged sidechains, eV","Ba interaction energy, charged sidechains, eV"
0,A,-0.138234,-0.118715,-0.100234,-0.138234,-0.118715,-0.100234
1,R,-3.225336,-2.127782,-1.599040,4.481403,4.390957,4.303165
2,N,-1.800469,-1.256246,-0.895895,-1.800469,-1.256246,-0.895895
3,D,-1.326600,-0.831851,-0.510109,-8.705812,-8.138319,-7.615657
4,C,-0.880283,-0.458234,-0.233128,-0.880283,-0.458234,-0.233128
5,Q,-2.364570,-1.721022,-1.321998,-2.364570,-1.721022,-1.321998
6,E,-1.717493,-1.120843,-0.900507,-8.961262,-8.330724,-7.762967
7,G,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,H,-2.353829,-1.601587,-1.211088,5.551417,5.389953,5.226539
9,I,-0.638260,-0.347956,-0.270345,-0.638260,-0.347956,-0.270345


In [10]:
res.to_csv('data/energies_final.csv', index=False)

In [11]:
# print for README
cols = ['Aminoacid'] + [f'E<sub>int</sub>({M}<sup>2+</sup>), eV' for M in ('Mg', 'Ca', 'Ba')] + \
       [f'E<sub>int</sub>({M}<sup>2+</sup>), charged sidechains, eV' for M in ('Mg', 'Ca', 'Ba')]
print('| ' + ' | '.join(cols) + ' |')
print('| ' + ' | '.join([':---:'] + ['----:']*(len(res.columns) - 1)) + ' |')
for row in res.to_dict('tight')['data']:
    line = '| ' + ' | '.join([row[0]] + [f'{c:>8.3f}' for c in row[1:]]) + ' |'
    print(line)

| Aminoacid | E<sub>int</sub>(Mg<sup>2+</sup>), eV | E<sub>int</sub>(Ca<sup>2+</sup>), eV | E<sub>int</sub>(Ba<sup>2+</sup>), eV | E<sub>int</sub>(Mg<sup>2+</sup>), charged sidechains, eV | E<sub>int</sub>(Ca<sup>2+</sup>), charged sidechains, eV | E<sub>int</sub>(Ba<sup>2+</sup>), charged sidechains, eV |
| :---: | ----: | ----: | ----: | ----: | ----: | ----: |
| A |   -0.138 |   -0.119 |   -0.100 |   -0.138 |   -0.119 |   -0.100 |
| R |   -3.225 |   -2.128 |   -1.599 |    4.481 |    4.391 |    4.303 |
| N |   -1.800 |   -1.256 |   -0.896 |   -1.800 |   -1.256 |   -0.896 |
| D |   -1.327 |   -0.832 |   -0.510 |   -8.706 |   -8.138 |   -7.616 |
| C |   -0.880 |   -0.458 |   -0.233 |   -0.880 |   -0.458 |   -0.233 |
| Q |   -2.365 |   -1.721 |   -1.322 |   -2.365 |   -1.721 |   -1.322 |
| E |   -1.717 |   -1.121 |   -0.901 |   -8.961 |   -8.331 |   -7.763 |
| G |    0.000 |    0.000 |    0.000 |    0.000 |    0.000 |    0.000 |
| H |   -2.354 |   -1.602 |   -1.211 |    5.551 |    5.390